In [1]:
import itertools
import time
from CliffordAlgebras import Clifford
import DiracOperator
import numba_DiracOperator
import numpy as np
import numba
from numba import jit, njit
# Some dummy parameters for use when testing:
p = 1
q = 3
N = 5
g2 = -3.7
g4 = 1
# Get the clifford algebra for the model we are examining.
cliff = Clifford(p, q)
cliff.introduce()
odd_products = cliff.get_odd_gamma_products()

My type is: (1,3)

My matrices will be 4x4, and I have K0 dimension s = 2
The generators are:
Gamma_0 = 
[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j -0.+0.j  0.+0.j -1.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j -0.+0.j]]
Gamma_1 = 
[[0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]]
Gamma_2 = 
[[ 0.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [-1.+0.j -0.+0.j  0.+0.j  0.+0.j]
 [-0.+0.j -1.+0.j  0.+0.j  0.+0.j]]
Gamma_3 = 
[[ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]
 [-0.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j -0.+0.j  0.+0.j]]


The chirality operator is:
[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [2]:
odd_products = np.ascontiguousarray(odd_products)

In [3]:
odd_products = odd_products.T

In [4]:
for prod in odd_products:
    print(numba.typeof(prod))

array(complex128, 2d, A)
array(complex128, 2d, A)
array(complex128, 2d, A)
array(complex128, 2d, A)


There are some import hyperparameters for the Metropolis-Hastings algorithm. Specifically,
in our implementation, the variable weightA is important, which is the step size between Dirac operators.
WeightA to be very small, so that the steps between Diracs is relatively small. There is a balance between having a small enough step size so that many Diracs are accepted and we "follow the valley downwards" and having a large enough step size so that we can escape local minima.

This value will change for each type, it seems to be a hyperparameter of this setup, i.e. requires tuning. weightA values for the types that give acceptance_rate/num_moves ~50% (this is what I understand to be a good thing to achieve)

(2,0) -> 1./np.power(cliff.matdim,3)/25
(1,3) -> 1./e-2/6 for size 10x10 - takes 30mins to do 40,000 moves.

In [5]:
# Initialise D: we set the weight A so we start with a much wide range of the parameter space

weightA = 1 / 10  # If we use too large a value here, then the proposed starting Dirac causes the weight action to have a large imaginary component. So start with it ~1
matdim = cliff.matdim
D = DiracOperator.random_Dirac_op(odd_products, N, weightA, cliff.matdim)
# D = D1


def runMonteCarlo(odd_products, D, g2, g4, matdim):
    weightA = 1e-2 / 7
    num_moves = 0
    acceptance_rate = 0
    epochs = 10
    chain_length = 1000
    for i in range(epochs):
        for j in range(chain_length):
            D, acceptance_rate = DiracOperator.update_Dirac(
                D, g2, g4, weightA, acceptance_rate, N, matdim, odd_products)
            num_moves += 1
        # Every 1000s steps, we investigate a little. We print the accepted rate and the action of the Dirac at this moment.
        print(acceptance_rate, num_moves, acceptance_rate / num_moves)
        print(DiracOperator.action(D, g2, g4))
        # np.linalg.eigvals(D)



UnboundLocalError: local variable 'Dtemp' referenced before assignment

In [6]:
D = numba_DiracOperator.random_Dirac_op(odd_products, N, weightA, cliff.matdim)
@njit()      
def numba_runMonteCarlo(odd_products, D, g2, g4, matdim):
    weightA = 1e-2 / 7
    num_moves = 0
    acceptance_rate = 0
    epochs = 10
    chain_length = 1000
    for i in range(epochs):
        for j in range(chain_length):
            D, acceptance_rate = numba_DiracOperator.update_Dirac(
                D, g2, g4, weightA, acceptance_rate, N, matdim, odd_products)
            num_moves += 1
        # Every 1000s steps, we investigate a little. We print the accepted rate and the action of the Dirac at this moment.
        print(acceptance_rate, num_moves, acceptance_rate / num_moves)
        print(numba_DiracOperator.action(D, g2, g4))
        # np.linalg.eigvals(D)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<function kron at 0x103e35c20>) found for signature:
 
 >>> kron(array(complex128, 2d, A), array(complex128, 2d, C))
 
There are 2 candidate implementations:
   - Of which 2 did not match due to:
   Overload in function 'kron_impl': File: numba/np/linalg.py: Line 2729.
     With argument(s): '(array(complex128, 2d, A), array(complex128, 2d, C))':
    Rejected as the implementation raised a specific error:
      TypingError: np.linalg.kron only supports 'C' or 'F' layout input arrays. Receieved an input of layout 'A'.
  raised from /Users/pauldruce/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numba/np/linalg.py:2672

During: resolving callee type: Function(<function kron at 0x103e35c20>)
During: typing of call at /Users/pauldruce/Documents/GitHub/RandomNCGinPython/numba_DiracOperator.py (89)


File "numba_DiracOperator.py", line 89:
def random_Dirac_op(odd_products, N, wei

In [ ]:
"""
It would be helpful to store the last Dirac, so we can "continue" the simulation after it reaches 20,0000 steps. For instance, for the type (1,3), 20,000 steps doesnt seem to be enough. 40,0000 seems to settle in a well.
But to have the thermalisation time be ~1%, then we need to continue the simulation for much longer than that.
"""

# We can then "initialise" the Dirac with the last Dirac operator from the simulation if we want to continnue the simulation.
D1 = D

In [7]:
# Test it works and see how long it takes

%time runMonteCarlo(odd_products, D, g2, g4, matdim)

778 1000 0.778
-153.1408222179947
1738 2000 0.869
-162.61195783746854
2687 3000 0.8956666666666667
-163.02613713281525
3634 4000 0.9085
-163.11108521848996
4573 5000 0.9146
-161.1350546679376
5519 6000 0.9198333333333333
-157.67950610551105
6455 7000 0.9221428571428572
-157.48070057510958
7406 8000 0.92575
-161.42429920827348
8340 9000 0.9266666666666666
-161.54910990958157
9284 10000 0.9284
-161.6646186058401
CPU times: user 28.7 s, sys: 402 ms, total: 29.1 s
Wall time: 29.1 s


In [5]:
# Test it works and see how long it takes

%time numba_runMonteCarlo(odd_products, D, g2, g4, matdim)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<function kron at 0x1059ffc20>) found for signature:
 
 >>> kron(array(complex128, 2d, A), array(complex128, 2d, C))
 
There are 2 candidate implementations:
   - Of which 2 did not match due to:
   Overload in function 'kron_impl': File: numba/np/linalg.py: Line 2729.
     With argument(s): '(array(complex128, 2d, A), array(complex128, 2d, C))':
    Rejected as the implementation raised a specific error:
      TypingError: np.linalg.kron only supports 'C' or 'F' layout input arrays. Receieved an input of layout 'A'.
  raised from /Users/pauldruce/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numba/np/linalg.py:2672

During: resolving callee type: Function(<function kron at 0x1059ffc20>)
During: typing of call at /Users/pauldruce/Documents/GitHub/RandomNCGi

In [4]:
# Profile it
%prun runMonteCarlo(odd_products, D, g2, g4, matdim)

867 1000 0.867
-151.53029253649206
1738 2000 0.869
-236.32358421850307
2561 3000 0.8536666666666667
-308.8966218233402
3420 4000 0.855
-334.21671711011237
4313 5000 0.8626
-336.78205153244784
5204 6000 0.8673333333333333
-339.2146358697222
6111 7000 0.873
-337.9076429029567
6988 8000 0.8735
-329.5990929038716
7870 9000 0.8744444444444445
-334.3759285810087
8755 10000 0.8755
-338.4953815775131
 

         13791394 function calls (12991394 primitive calls) in 33.946 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
1220040/420040    8.007    0.000   23.774    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   240000    5.243    0.000    5.997    0.000 numeric.py:824(outer)
    10000    2.599    0.000   30.639    0.003 DiracOperator.py:68(random_Dirac_op)
   240000    2.590    0.000   21.126    0.000 shape_base.py:1067(kron)
    20010    1.906    0.000    2.701    0.000 DiracOperator.py:93(action)
    80000    1.327    0.000    2.110    0.000 DiracOperator.py:57(random_Hermitian)
   160000    0.819    0.000    1.167    0.000 twodim_base.py:152(eye)
   480000    0.801    0.000    2.128    0.000 {built-in method builtins.sorted}
   100000    0.664    0.000    1.588    0.000 numeric.py:2317(array_equal)
    60000    0.648    0.000   10.395    0.000 DiracOperator.py:42(anticomm)
   160000    0.

In [8]:
%load_ext line_profiler

In [11]:
# Profile it
 runMonteCarlo(odd_products, D, g2, g4, matdim)

821 1000 0.821
-282.2733852259223
1708 2000 0.854
-335.4615902727121
2598 3000 0.866
-334.53251904087597
3502 4000 0.8755
-339.78751494096366
4402 5000 0.8804
-339.9971988078137
5308 6000 0.8846666666666667
-336.1104997663529
6203 7000 0.8861428571428571
-337.0768581853198
7104 8000 0.888
-331.40436871663553
8025 9000 0.8916666666666667
-335.81635970418597
8938 10000 0.8938
-341.2391831535927


Timer unit: 1e-06 s

Total time: 0 s
File: /Users/pauldruce/Documents/GitHub/RandomNCGinPython/DiracOperator.py
Function: update_Dirac at line 119

Line #      Hits         Time  Per Hit   % Time  Line Contents
   119                                           @njit(fastmath=True)
   120                                           def update_Dirac(old_D, g2, g4, weightA, acceptance_rate, N, matdim, odd_products):
   121                                               """ Updates the Dirac operator according to the Metropolis-Hastings algorithm
   122                                           
   123                                               This function inputs the old Dirac operator, the action coupling constants, weightA which dictates the steps size between the old and new Dirac operators, and a variable to calculate the acceptance_rate. This function could be done smoother I'm sure. But it seems to work.
   124                                           
   125                       